In [1]:
import pandas as pd
from itertools import product
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
pd.set_option('display.max_columns', None)

In [2]:
original = pd.read_csv('../html2023-spring-final-project/train.csv')

In [3]:
trainingLabels = original['Danceability']
original.drop(['Danceability'], axis=1, inplace=True) 

In [4]:
def fillOptions(data, option = 'max'):
    if data.isna().sum() != len(data):
        if option == 'max':
            return data.value_counts().idxmax()
        elif option == 'mean':
            return data.mean()
        elif option == 'median':
            return data.median()

In [5]:
def filterArtistComposerDance(data, nameColumnFill):

    listArtist = data['Artist'].unique()
    listComposer = data['Composer'].unique()
    
    filter = list(product(listArtist, listComposer))

    newData = pd.DataFrame(columns=data.columns)

    for i in filter:
        artist, composer = i[0], i[1]
        filterData = data[(data['Artist'] == artist) & (data['Composer'] == composer) ].copy()
        if len(filterData) != 0:
            # Fill column Name
            for nameColumn in nameColumnFill:
                fillInfo = fillOptions(filterData[nameColumn], option = 'max')
                if fillInfo != None:
                    filterData.loc[:,nameColumn].fillna(fillInfo, inplace=True)

            newData = pd.concat([newData, filterData], ignore_index=True)
    return newData

In [6]:
def filterArtist(data, nameColumnFill):
    listDance = data['Artist'].unique()
    filter = listDance


    newData = pd.DataFrame(columns=data.columns)

    for i in filter:
        dance = i
        filterData = data[ (data['Artist'] == dance)].copy()
    
    
        if len(filterData) != 0:
            # Fill column Name
            for nameColumn in nameColumnFill:
                fillInfo = fillOptions(filterData[nameColumn], option = 'max')
                if fillInfo != None:
                    filterData.loc[:,nameColumn].fillna(fillInfo, inplace=True)

            newData = pd.concat([newData, filterData], ignore_index=True)

    return newData

In [7]:
def filterFillData(data, nameColumnFill):
    if not data.isnull().any().any():
        return data
    else:
        for nameColumn in nameColumnFill:
            fillInfo = fillOptions(data[nameColumn], option = 'max')
            if fillInfo != None:
                data.loc[:,nameColumn].fillna(fillInfo, inplace=True)
    return data



In [8]:
def dataPreprocessing(original):
    
    # pd.options.mode.chained_assignment = None

    data = original.copy()

    nameColumnFill = ['Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'Duration_ms', 'Views', 'Likes', "Stream" , "Comments"]

    # License and official_video
    data['Licensed'].fillna(data['official_video'], inplace=True)
    data['Licensed'].fillna(False, inplace=True)

    data['official_video'].fillna(data['Licensed'], inplace=True)
    data['official_video'].fillna(False, inplace=True)

    data['official_video'].fillna(False, inplace=True)
    data['Licensed'].fillna(False, inplace=True)

    data['Licensed'] =  data['Licensed'].map({True: 1, False: 0})
    data['official_video'] = data['official_video'].map({True: 1, False: 0})
    
    # Create new class = 'Unknown'
    data['Composer'].fillna("Unknown", inplace=True)
    data['Artist'].fillna("Unknown", inplace=True)
    data['Album_type'].fillna("Unknown", inplace=True)

    newData = filterArtistComposerDance(data, nameColumnFill)
    data = newData.copy()

    newData = filterArtist(data, nameColumnFill)
    data = newData.copy()

    newData = filterFillData(data, nameColumnFill)
    data = newData.copy()

    #Transform type key to use as class
    data['Key'] = data['Key'].astype(int)
    data['Key'] = data['Key'].astype(str)

    data = data.sort_values('id')

    # DELETE Track, Album, Uri, Url_spotify, Url_youtube, Description, Title, Channel, id, Comments
    data.drop(['Track', 'Album', 'Uri', 'Url_spotify', 'Url_youtube', 'Description', 'Title', 'Channel', 'id'], axis=1, inplace=True)

    # pd.options.mode.chained_assignment = 'warn'

    return data
    


In [9]:
def convertEncoderPD(data, prefix = 'key'):
    titleKeys = []
    for i in range(data.shape[1]):
        titleKeys.append(f'{prefix}_{i}')
    
    return pd.DataFrame(data=data, columns= titleKeys)


In [10]:
# minX -60  maxX = 0  ~ 0 - 1
# y = (-1/60) x
def scaleMinMaxLoudness(data):
    return -data/60

In [27]:
def createEncodeDataTraining(data):

    encoderKey = OneHotEncoder()
    encodedKey = encoderKey.fit_transform(data[['Key']])
    Key = encodedKey.toarray()
    key_pd = convertEncoderPD(Key, prefix = 'key')

    encoderAlbumType = OneHotEncoder()
    encodedKeyAlbumType = encoderAlbumType.fit_transform(data[['Album_type']])
    AlbumType = encodedKeyAlbumType.toarray()
    AlbumType_pd = convertEncoderPD(AlbumType, prefix = 'AlbumType')

    encoderComposer = OneHotEncoder()
    encodedKeyComposer = encoderComposer.fit_transform(data[['Composer']])
    Composer = encodedKeyComposer.toarray()
    Composer_pd = convertEncoderPD(Composer, prefix = 'Composer')

    encoderArtist = OneHotEncoder()
    encodedArtist = encoderArtist.fit_transform(data[['Artist']])
    Artist =  encodedArtist.toarray()
    Artist_pd = convertEncoderPD(Artist, prefix = 'Artist')


    # encoderArtist = LabelEncoder()
    # encodedArtist = encoderArtist.fit_transform(data[['Artist']])
    # # encodedArtist = encodedArtist.ravel()
    # Artist_pd =  pd.DataFrame(data=encodedArtist, columns= ["Artist"])

    data.drop(['Key','Album_type', 'Composer',  'Artist'], axis=1, inplace=True)

    data = pd.concat([data, key_pd, AlbumType_pd, Composer_pd, Artist_pd], axis=1)


    scaledLoudness = scaleMinMaxLoudness(data[['Loudness']])
    data['Loudness'] = scaledLoudness

    newMinMaxScaler = ['Tempo', 'Duration_ms', 'Views', 'Likes', 'Stream', 'Comments']

    scaler = StandardScaler()
    scaledData = scaler.fit_transform(data[newMinMaxScaler])

    for i in range(scaledData.shape[1]):
        data[newMinMaxScaler[i]] = scaledData[:, i]


    return {"key": encoderKey, 'AlbumType': encoderAlbumType, 'Composer': encoderComposer, "Artist":encoderArtist} , scaler, data
    

In [23]:
def createEncodeDataTesting(encoder, scalerStandard, data):

    encoderKey = encoder['key']
    encodedKey = encoderKey.transform(data[['Key']])
    Key = encodedKey.toarray()
    key_pd = convertEncoderPD(Key, prefix = 'key')

    encoderAlbumType = encoder['AlbumType']
    encodedKeyAlbumType = encoderAlbumType.transform(data[['Album_type']])
    AlbumType = encodedKeyAlbumType.toarray()
    AlbumType_pd = convertEncoderPD(AlbumType, prefix = 'AlbumType')

    encoderComposer = encoder['Composer']
    encodedKeyComposer = encoderComposer.transform(data[['Composer']])
    Composer = encodedKeyComposer.toarray()
    Composer_pd = convertEncoderPD(Composer, prefix = 'Composer')   

    encoderArtist = encoder['Artist']
    encodedArtist = encoderArtist.transform(data[['Artist']])
    encodedArtist = encodedArtist.ravel()
    Artist_pd =  pd.DataFrame(data=encodedArtist, columns= ["Artist"])

    data.drop(['Key','Album_type', 'Composer',  'Artist'], axis=1, inplace=True)

    data = pd.concat([data, key_pd, AlbumType_pd, Composer_pd, Artist_pd], axis=1)

    scaledLoudness = scaleMinMaxLoudness(data[['Loudness']])
    data['Loudness'] = scaledLoudness

    newMinMaxScaler = ['Tempo', 'Duration_ms', 'Views', 'Likes', 'Stream', 'Comments']

    scaler = scalerStandard
    scaledData = scaler.transform(data[newMinMaxScaler])

    for i in range(scaledData.shape[1]):
        data[newMinMaxScaler[i]] = scaledData[:, i]

    return data

#### Data Preparation

In [28]:
data = dataPreprocessing(original)

In [29]:
encoders, scalerStandard, scaledData_pd  = createEncodeDataTraining(data)

In [30]:
scaledData_pd


,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,Likes,Stream,Licensed,official_video,Comments,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,AlbumType_0,AlbumType_1,AlbumType_2,AlbumType_3,Composer_0,Composer_1,Composer_2,Composer_3,Composer_4,Composer_5,Composer_6,Composer_7,Composer_8,Composer_9,Composer_10,Artist_0,Artist_1,Artist_2,Artist_3,Artist_4,Artist_5,Artist_6,Artist_7,Artist_8,Artist_9,Artist_10,Artist_11,Artist_12,Artist_13,Artist_14,Artist_15,Artist_16,Artist_17,Artist_18,Artist_19,Artist_20,Artist_21,Artist_22,Artist_23,Artist_24,Artist_25,Artist_26,Artist_27,Artist_28,Artist_29,Artist_30,Artist_31,Artist_32,Artist_33,Artist_34,Artist_35,Artist_36,Artist_37,Artist_38,Artist_39,Artist_40,Artist_41,Artist_42,Artist_43,Artist_44,Artist_45,Artist_46,Artist_47,Artist_48,Artist_49,Artist_50,Artist_51,Artist_52,Artist_53,Artist_54,Artist_55,Artist_56,Artist_57,Artist_58,Artist_59,Artist_60,Artist_61,Artist_62,Artist_63,Artist_64,Artist_65,Artist_66,Artist_67,Artist_68,Artist_69,Artist_70,Artist_71,Artist_72,Artist_73,Artist_74,Artist_75,Artist_76,Artist_77,Artist_78,Artist_79,Artist_80,Artist_81,Artist_82,Artist_83,Artist_84,Artist_85,Artist_86,Artist_87,Artist_88,Artist_89,Artist_90,Artist_91,Artist_92,Artist_93,Artist_94,Artist_95,Artist_96
0,0.000273,0.419600,0.0443,7.241508e-01,0.000062,0.000807,0.3400,-1.255897,-0.929988,-0.152250,-0.281016,-0.449627,0,0,-0.110374,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,0.184220,0.229350,0.0340,6.722214e-01,0.910000,0.034966,0.7460,0.944228,-0.447282,-0.352389,-0.363223,-0.485495,0,0,-0.125275,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
910,0.384241,0.259933,0.0442,6.676276e-01,0.867000,0.001772,0.3800,-1.205320,-0.645164,-0.352389,-0.363223,-0.452510,0,0,-0.125275,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1121,0.209585,0.104183,0.0277,3.796416e-03,0.000000,0.001000,0.5110,0.995715,0.348878,0.494190,0.252463,1.207829,1,1,-0.002817,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1367,0.354895,0.082267,0.0260,3.048625e-06,0.000467,0.002924,0.2950,-0.763974,0.339028,0.972284,0.439960,-0.470537,1,1,0.031552,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [16]:
train_X, test_X, train_Y, test_Y = train_test_split(scaledData_pd, trainingLabels, test_size = 0.20, random_state = 123)
train_X, Validation_X, train_Y, Validation_Y = train_test_split(train_X, train_Y, test_size = 0.20, random_state = 123)

### SVM grid search

In [17]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# Create the GradientBoostingClassifier object
clf = GradientBoostingClassifier()

# Create the grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'subsample': [0.8, 0.9, 1.0],
}

# Create the grid search object
grid_search = GridSearchCV(clf, param_grid, cv=5)

# Fit the grid search object to the data
grid_search.fit(train_X, train_Y)

# Print the best parameters
print(grid_search.best_params_)

# Print the best score

KeyboardInterrupt: 

In [31]:
# clf1 = SVC(kernel='linear',  gamma=0.001, C=100)
# clf1.fit(train_X, train_Y)

# filename = "best_SVM.model"
# pickle.dump(clf1, open(filename, "wb"))

In [32]:
# # load model
# loaded_model = pickle.load(open(filename, "rb"))
# # you can use loaded model to compute predictions
# predicted = loaded_model.predict(test_X)
# # predicted = np.rint(predicted)

# errors = abs(predicted - test_Y)
# # Print out the mean absolute error (mae)
# print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 2.33


#### Load Testing Dataset

In [33]:
# test = pd.read_csv('../html2023-spring-final-project/test.csv')
# dataTest = dataPreprocessing(test)
# testScaled = createEncodeDataTesting(encoders, scalerStandard, dataTest)

c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Testing Danceability

In [34]:
# # load model
# loaded_model_realTest= pickle.load(open(filename, "rb"))
# # you can use loaded model to compute predictions
# predictedTest = loaded_model_realTest.predict(testScaled)
# predictedTest = np.rint(predictedTest)

In [35]:
# label = 17170
# idx = []
# for i in range(predictedTest.shape[0]):
#     idx.append(label + i)
# idx = np.array(idx)

In [36]:
# testR = np.vstack((idx,predictedTest))
# testR = testR.T

In [37]:
# predictionTest_pd = pd.DataFrame(data = testR, columns= ['id', 'Danceability'])
# predictionTest_pd.to_csv('SVM2.csv')